In [2]:
import pandas as pd

# Chemin du fichier Excel
file_path = "../sources/donnéesSorties.xlsx"

# Lire le fichier Excel
df = pd.read_excel(file_path)

# Afficher les premières lignes du tableau
print(df)

# Exploiter les données (exemple : calculer la moyenne du chiffre d'affaires)
#moyenne_chiffre_affaires = df["Chiffre d Affaires (k€)"].mean()
#print(f"Moyenne du chiffre d'affaires (en k€) : {moyenne_chiffre_affaires:.2f}")

# Transformer le tableau en dictionnaire si besoin
data_dict = df.to_dict(orient="records")
print(data_dict)


   LIB_STR_NIV2 LIB_STR_NIV3  ID_SEM_COMM     CONSO_DEC  OBJ_CONSO_DEC  \
0      BANQUE A        GRP 1       202446  2.393425e+06   2.364556e+06   
1      BANQUE B        GRP 2       202446  4.722818e+06   4.729637e+06   
2      BANQUE C        GRP 3       202446  1.603872e+06   1.692898e+06   
3      BANQUE D        GRP 1       202446  3.119778e+06   3.035599e+06   
4      BANQUE E        GRP 2       202446  1.939443e+06   2.112013e+06   
..          ...          ...          ...           ...            ...   
69    BANQUE BR        GRP 4       202446  5.108356e+06   6.241839e+06   
70    BANQUE BS        GRP 3       202446  2.131199e+06   2.326228e+06   
71    BANQUE BT        GRP 7       202446  2.214313e+06   2.683143e+06   
72    BANQUE BU        GRP 7       202446  1.696758e+06   3.168115e+06   
73    BANQUE BV        GRP 3       202446  2.394812e+06   3.902082e+06   

    TX_CONSO_DEC  RG_CONSO_DEC  OCC_PART  OBJ_OCC_PART  TX_OCC_PART  ...  \
0       0.892756            29     

In [3]:
import openai
import os
from dotenv import load_dotenv
import pandas as pd

# Charger la clé API depuis le fichier .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Fonction pour envoyer une requête avec un pre-prompt
def ask_gpt_with_preprompt(pre_prompt, user_query, model="gpt-4-0613", max_tokens=500):
    try:
        # Messages comprenant le pre-prompt et la question utilisateur
        messages = [
            {"role": "system", "content": pre_prompt},  # Pre-prompt
            {"role": "user", "content": user_query}    # Question utilisateur
        ]
        
        # Requête à l'API
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.7  # Ajuste la créativité du modèle
        )
        
        # Retourne la réponse du modèle
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Erreur lors de l'appel à l'API : {e}"

# Pre-prompt pour guider GPT-4
pre_prompt = (
    "Tu es un analyste financier spécialisé dans le domaine bancaire. "
    "Tu vas analyser les données hebdomadaires de différentes agences pour une banque spécifique et rédiger un compte rendu clair et synthétique. "
    "Tu vas devoir analyser plusieurs champs en fonction de leurs définition. "
    "Dans cette banque nous différencions les clients particuliers qui sont représenté par les gens normaux avec leurs comptes personnels et les clients professionels qui sont des entreprises de tailles modetestes (auto-entrepreneur, SARL, SASU)"
)

# Générer un compte rendu pour chaque banque
def generate_reports(df):
    reports = {}
    for index, row in df.iterrows():
        # Assignation temporaire des valeurs pour éviter les conflits avec les f-strings 
        bank_name = row["LIB_STR_NIV2"]
        groupe_name = row["LIB_STR_NIV3"]
        num_semaine = row["ID_SEM_COMM"]
        occ_part = row["OCC_PART"]
        conso_dec = row["CONSO_DEC"]
        bilan_net = row["BILAN_NET"]
        vers_ep_fi = row["FI_NET"]
        vb_iard_part = row["VB_IARD_PART"]
        vb_prev_part = row["VB_PREV_PART"]
        vb_ass_vie_peri_pea_hisseo = row["VB_ASS_PERI_PEA_HISS"]
        moy_part = row["MOY_TX_PART"]
        somme_part = row["S_RG_PART"]
        rang_part = row["RANG_PART"]
        occ_pro = row["OCC_PRO"]
        occ_pro_cible = row["OCC_PRO_CIBLE"]
        eqpt_dec = row["EQPT_DEC"]
        vb_iard_pro = row["VB_IARD_PRO"]
        vb_prev_pro = row["VB_PREV_PRO"]
        vb_contr_comm = row["VB_CONTR_COMM"]
        moy_pro = row["MOY_TX_PRO"]
        somme_pro = row["S_RG_PRO"]
        rang_pro = row["RANG_PRO"]
        moy = row["MOY"]
        somme_rang = row["S_RG"]
        rang = row["RANG"]

        # Construire la question utilisateur à partir des données de la banque
        user_query = (
            f"Voici les données de la banque {bank_name} de la semaine {num_semaine} :\n"
            f"- Nombre d'ouverture de compte conventionné pour la clientèle particulière : {occ_part}\n"
            f"- La production de prêt de consommation décaissé. Lorsqu'aun prêt est accordé, l'argent n'est pas directement versé, elle l'est au moment du décaisement. : {conso_dec} k€\n"
            f"- La collecte de Bilan nette, y compris les dépôts à vue. Aussi appelé collecte bilantielle est la somme des flux créditeur et débiteur par agence : {bilan_net} k€\n"
            f"- Le versement épargne fiancier représente tous les produits d'épargne à rendement de titres (Plan d'Epargne en Action, Compte Titre Ordinaire, Plan d'Epargne pour la Retraite COllectif, Plan Epargne Entreprise, Livret de Développement Durable, Livret d'Epargne Populaire, Livret A) : {vers_ep_fi} k€\n"
            f"- Le nombre de ventes brutes des produits d'assurances (Incendie, Accidents, risques divers) pour les particuliers : {vb_iard_part} \n"
            f"- Le nombre de ventes brutes des produits de prévoyance (Assurances Décès, maladie, ect...) pour les particuliers  : {vb_prev_part} \n"
            f"- Le nombre de ventes brutes des assurances vie, des plan d'Epargne en Actions, des PERI (type de plan d'Epargne Retraite) et des Hisséo (autre type d'assurance vie) : {vb_ass_vie_peri_pea_hisseo} \n"
            f"- La moyenne du taux d'atteinte pour les particuliers en fonction des objectifs : {moy_part} %\n"
            f"- La somme des rangs des différentes catégorie dédié aux particuliers : {somme_part} \n"
            f"- Position de l'agence par rapport aux autres agences dans le segment des clients particuliers : {rang_part} \n"
            f"- Nombre d'ouverture de compte conventionné pour la clientèle professionnelle : {occ_pro} \n"
            f"- Nombre d'ouverture de comptes conventionné pouf la clientèle professionnelle ciblé (Profession libéral, professionnel installé depuis plus de 2 ans) : {occ_pro_cible} \n"
            f"- La production de crédit d'équipement décaissé : {eqpt_dec} k€\n"
            f"- Le nombre de ventes brutes des produits d'assurances (Incendie, Accidents, risques divers) pour les professionnelles : {vb_iard_pro} \n"
            f"- Le nombre de ventes brutes des produits de prévoyance (Assurances Décès, maladie, ect...) pour les professionnelles : {vb_prev_pro} \n"
            f"- Le nombre de ventes brutes des contrats commerçants  : {vb_contr_comm} \n"
            f"- La moyenne du taux d'atteinte pour les professionnelles en fonction des objectifs : {moy_pro} %\n"
            f"- La somme des rangs des différentes catégorie dédié aux professionnelles : {somme_pro} \n"
            f"- Position de l'agence par rapport aux autres agences dans le segment des clients professionnelles : {rang_pro} \n"
            f"- La moyenne des moyennes des particuliers et des progessionnels : {moy} %\n"
            f"- La somme des sommes des rangs des particuliers et des professionnelles par agence: {somme_rang} \n"
            f"- Le rang final de l'agence est calculé en faisant la somme des rangs des 13 items particuliers et professionnelles : {rang} \n"

            "Rédige un compte rendu détaillé et pertinent pour cette agence, qui s'adresse à son directeur d'agence."
        )
        
        # Appeler GPT pour générer le compte rendu
        report = ask_gpt_with_preprompt(pre_prompt, user_query)
        reports[bank_name] = report
    
    return reports

# Générer les comptes rendus
reports = generate_reports(df)

# Afficher les comptes rendus
for agence, report in reports.items():
    print(f"\nCompte rendu pour {agence} :\n{report}\n")


KeyboardInterrupt: 